202. Happy Number

https://leetcode.cn/problems/happy-number/

## 方案一：利用哈希集合检测循环

### 核心思想
解决此问题的关键在于对“快乐数”计算过程的观察。其核心操作是“求各位数字的平方和”，这个操作对于足够大的数而言，是一个**快速衰减**的过程。

可以证明，任何一个大于 243 的数，经过一次“各位数字平方和”的计算后，得到的结果必然比其自身要小。（注：一个更严格的结论是，该性质对于所有大于等于 100 的数都成立，此处证明省略）。

基于这一观察，可以推导出任何一个正整数的计算序列都只有两种最终结局：
1.  **最终等于 1**：计算序列收敛，该数为快乐数。
2.  **永远不等于 1**：由于任何大的数字都会迅速衰减到一个有限的区间内，那么一个永不为 1 的序列必然会停留在这个有限的数字集合中。根据**鸽巢原理**，在一个有限的集合中无限地取数，必然会取到重复的数字，即进入一个**无限循环**。

这个“必然进入循环”的结论，是采用哈希集合（`HashSet`）来解决此问题的理论基础。通过记录计算过程中出现过的数字，就可以在 O(1) 的平均时间复杂度内判断是否出现了重复，从而有效地检测到循环。

### 算法思路
1.  初始化一个哈希集合 `seen`，用于存储计算序列中出现过的所有数字。
2.  启动一个循环，其继续的条件是：当前数字 `n` 不等于 1，且 `n` 尚未出现在 `seen` 集合中。
3.  在每次循环的开始，将当前的数字 `n` 添加到 `seen` 集合中。
4.  计算 `n` 的各位数字平方和，并将结果作为新的 `n`，用于下一次循环。
5.  当循环终止时，判断 `n` 的值。如果 `n` 等于 1，则原始数字是快乐数；否则，说明陷入了循环。

### 快乐数的数学结论
通过理论分析与有限范围内的穷举验证，可以得出关于十进制下快乐数的几个确定性结论：
* **有界性**：任何正整数的计算序列最终都会落入一个有界的数值区间内（例如 `[1, 243]`）。
* **必然循环性**：任何不以 1 结尾的序列，最终必然会进入一个循环。
* **循环唯一性**：在十进制中，所有非快乐数最终都会汇入同一个循环：`4 → 16 → 37 → 58 → 89 → 145 → 42 → 20 → 4`。快乐数则最终进入 `1 → 1` 这个循环。

### 复杂度分析 
#### 时间复杂度: $O(\log n)$
算法的总时间是其执行的所有“求各位平方和”操作的耗时总和。可将计算序列分为两个阶段进行分析：

1.  **骤降阶段的耗时分析**
    此阶段指数字从初始值 `n` 降至一个有界常数范围的过程。设初始数字为 $n_0 = n$，计算序列为 $n_1 = \text{next}(n_0), n_2 = \text{next}(n_1), \ldots$。

    * **第一次计算**: 计算 $n_1 = \text{next}(n_0)$。该操作的耗时与 $n_0$ 的位数成正比，因此时间成本为 $O(\log n_0)$。计算结果 $n_1$ 的值最大不超过 $81 \times (\lfloor\log_{10}n_0\rfloor + 1)$，因此 $n_1$ 的数值大小量级为 $O(\log n_0)$。

    * **第二次计算**: 计算 $n_2 = \text{next}(n_1)$。由于 $n_1$ 的数值大小量级为 $O(\log n_0)$，其位数与 $\log(n_1)$ 成正比，即与 $\log(\log n_0)$ 成正比。因此，本次计算的时间成本为 $O(\log(\log n_0))$。

    * **后续计算**: 以此类推，第 `i` 次计算的耗时为 $O(\log n_{i-1})$，其结果在数值上呈对数级别快速衰减。

    整个骤降阶段的总时间是所有这些计算耗时的总和：
    $$
    T_{\text{骤降}} = O(\log n) + O(\log(\log n)) + O(\log(\log(\log n))) + \ldots
    $$
    这是一个收敛速度极快的级数。在复杂度分析中，第一项 $O(\log n)$ 的增长速度远超后续所有项，是绝对的主导项。因此，整个骤降阶段的时间复杂度就是 $O(\log n)$。

2.  **循环检测阶段的耗时分析**
    一旦数字落入有界的常数范围，后续的每一次计算耗时均为常数 $O(1)$。到达 1 或找到循环所需的计算次数也是一个常数。因此，此阶段的总时间是 $O(1)$。

**综合分析**：
算法的总时间复杂度是 $T_{\text{总}} = O(\log n) + O(1) = O(\log n)$。该复杂度主要由处理初始大数 `n` 的第一次计算所贡献。

#### 空间复杂度: $O(1)$ 与骤降次数的估计
空间复杂度由哈希集合 `seen` 的大小决定。关键在于确定**骤降阶段的计算次数**。

这个次数并没有一个基于 `n` 的数值大小的严格递减公式，它更取决于 `n` 的**数字构成**。例如，一个形如 `100...0` 的数与一个形如 `99...9` 的数在量级上可能非常接近，但前者的骤降次数为 1（直接变为 1），而后者的骤降路径则更长。

尽管如此，可以证明，任何能在计算机中表示的数字，其骤降次数都极少，都可以被视为一个很小的常数。

因此，`seen` 集合中存储的元素总数由两部分构成：
1.  **骤降阶段**：存储一个极小常数个数的数字。
2.  **循环检测阶段**：存储一个常数个数的数字。

骤降阶段的计算次数，在很多情况下甚至会少于后续在小范围内寻找循环或终点 1 所需的计算次数。

**综合分析**：
由于 `seen` 集合需要存储的元素总数有一个不依赖于初始 `n` 大小的固定上限，因此算法所需的额外空间是一个常数，**空间复杂度为 $O(1)$**。

In [ ]:
class Solution1:
    def isHappy(self, n: int) -> bool:
        seen = set()
        while n != 1 and n not in seen:
            seen.add(n)
            n = self.get_next(n)
        return n == 1

    def get_next(self, n: int) -> int:
        total_sum = 0
        while n > 0:
            digit = n % 10
            total_sum += digit * digit
            n = n // 10
        return total_sum
                

## 方案二：快慢指针法 (Floyd 判环算法)

### 核心思想
此方法将“快乐数”问题巧妙地抽象成一个经典的计算机科学问题：**判断链表是否有环**。

可以将数字的计算序列看作一个概念上的“链表”：
* **节点 (Node)**：序列中的每一个数字 `n`。
* **后继指针 (Next Pointer)**：从 `n` 指向 `get_next(n)` 的计算过程。

基于这个模型，问题转化为：
1.  如果一个数是**快乐数**，那么这个“链表”最终会指向节点 `1` 并终止（或进入 `1->1` 的自循环）。
2.  如果一个数**不是快乐数**，那么这个“链表”最终必然会进入一个不包含 `1` 的环。

**Floyd 判环算法**（也称“龟兔赛跑算法”）是解决链表判环问题的标准方案。该算法设置两个指针，一个慢（一次走一步）一个快（一次走两步）。如果链表中存在环，快指针最终必然会从后面追上并与慢指针相遇。

### 算法思路
1.  初始化两个指针，`slow` 指向初始数字 `n`，`fast` 指向 `n` 的下一个数字 `get_next(n)`。
2.  启动一个循环，其继续的条件是：快指针 `fast` 不等于 1（未到达终点），且 `slow` 和 `fast` 不相等（尚未在环中相遇）。
3.  在每次循环中，将 `slow` 向前移动一步 (`slow = get_next(slow)`)，将 `fast` 向前移动两步 (`fast = get_next(get_next(fast))`)。
4.  当循环终止时，判断 `fast` 的值。如果 `fast` 等于 1，则原始数字是快乐数；否则，说明 `slow` 与 `fast` 相遇，陷入了循环。

### 复杂度分析
#### 时间复杂度: $O(\log n)$
快慢指针法的时间复杂度分析与哈希集合法类似。总的耗时依然由 `get_next` 函数的调用次数和单次耗时决定。
* **骤降阶段**: 对于一个极大的初始数 `n`，其序列值会快速下降。此阶段的耗时主要由第一次计算决定，为 $O(\log n)$。
* **循环检测阶段**: 一旦数字进入常数范围内的有界区间，Floyd 算法找到环或终点 1 所需的步数是一个与 `n` 无关的常数。
* **综合分析**: 算法的总时间复杂度由主导的骤降阶段决定，因此为 $O(\log n)$。

#### 空间复杂度: $O(1)$
这是快慢指针法相较于哈希集合法的最大优势。
该算法只使用了 `slow` 和 `fast` 等有限几个变量来存储中间状态，无论初始数字 `n` 有多大，所需的额外空间都是固定的。因此，其空间复杂度为**常数级 $O(1)$**。

In [ ]:
class Solution2:
    def isHappy(self, n: int) -> bool:
        # slow 从 n 开始，fast 从 n 的下一步开始
        # 这样可以避免初始时 slow == fast 导致循环无法进入
        slow = n
        fast = self.get_next(n)

        # 当 fast 不为 1 且 slow 和 fast 不相遇时，循环继续
        while fast != 1 and slow != fast:
            slow = self.get_next(slow)
            fast = self.get_next(self.get_next(fast))
        
        # 循环结束后，当且仅当 fast 等于 1 时，是快乐数
        return fast == 1
    
    def get_next(self, n: int) -> int:
        total_sum = 0
        while n > 0:
            digit = n % 10
            total_sum += digit * digit
            n = n // 10
        return total_sum